## Capstone Project 1
### Student: Donwell Tlou
**(Module - Data Science, Data Analysis and Data Visualization)**

### Problem Statement:
- Analyse given dataset using Python libraries like Numpy, Pandas, Matplotlib, Seaborn, and
sklearn. The given project targets to pre-process the dataset by applying feature engineering,
feature selection and exploratory data analysis to it. Actual model building is not expected as
a part of this project.

**Bank Dataset**
- The goal of the car price prediction dataset is to predict the price of a car based on various
features such as model, production year, category, brand, fuel type, engine volume, mileage,
cylinders, vehicle style, and others. The dataset consists of 19,237 samples and we need to use
80% data for training and using it for feature selection. The ultimate aim of this dataset is
developing machine learning models that can accurately predict car prices based on these
diverse features. This task is valuable for consumers looking to make informed decisions about
purchasing cars within their budget while maximizing the features they desire.

## Solution

## Section 1

**a. Analyse data types of features and verify they hold data same as that of their datatype. Update if required. Process ‘Levy’ column and convert it to integer type.**

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('price_prediction_batch_23.csv')

# Display basic information about the dataset
print(df.head())
print(df.info())

# Display data types of features
print(df.dtypes)

# Process 'Levy' column
df['Levy'] = df['Levy'].replace('-', '0')  # Replace any occurences of '-' with '0'
df['Levy'] = df['Levy'].str.replace(',', '').astype(int)  # Remove commas and convert to integer

# Display the cleaned DataFrame
print(df)


**b. Process ‘Mileage’ column and convert it to integer. Check for Nan values in data and remove them using appropriate method, if need be.**

In [ ]:
# Process 'Mileage' column
df['Mileage'] = df['Mileage'].str.replace(' km', '').str.replace(',', '').astype(int)  # Remove ' km' and commas, convert to integer

# Process 'Engine volume' column
# df['Engine volume'] = df['Engine volume'].apply(lambda x: float(re.search(r'(\d+\.?\d*)', x).group(1)) if re.search(r'(\d+\.?\d*)', x) else None)

# Drop rows with NaN values in 'Engine volume' column
df = df.dropna(subset=['Engine volume'])

# Check for NaN values
print(df.isnull().sum())

# Drop rows with NaN values
df = df.dropna()

**Conclusions/Observations:**

1. Data types have been verified and updated where necessary.
2. The `Levy` column has been successfully converted to integer type.
3. The `Mileage` column has been converted to integer, and NaN values have been removed.

## Section 2

**a. Check for duplicates, view duplicated rows, and remove them, if any.**

In [ ]:
# Check for duplicates
duplicates = df[df.duplicated()]

# View duplicated rows
print(duplicates)

# Remove duplicates
df = df.drop_duplicates()

**b. Check for outliers using boxplot and statistical method, and remove them, if any.**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Check for outliers using boxplot
sns.boxplot(x=df['Price'])
plt.show()

# Check for outliers using statistical method (e.g., Z-score)
from scipy import stats

z_scores = stats.zscore(df['Price'])
outliers = (z_scores > 3) | (z_scores < -3)

# Remove outliers
df = df[~outliers]

**c. For categorical features, draw countplot. Write your observations.**

In [ ]:
# Draw countplot for categorical features
categorical_features = [
    'Manufacturer', 'Model', 'Category', 'Leather interior', 
    'Fuel type', 'Gear box type', 'Drive wheels', 'Wheel', 'Color'
]
for feature in categorical_features:
    sns.countplot(x=feature, data=df)
    plt.title(f'Countplot of {feature}')
    plt.show()

**d. For numeric features, draw a histogram. Compute and about skewness of variables and apply transformation function, if needed.**

In [ ]:
# Draw histogram for numeric features
numeric_features = [
    'Price', 'Levy', 'Prod. year', 'Engine volume', 
    'Mileage', 'Cylinders', 'Airbags'
]
for feature in numeric_features:
    sns.histplot(df[feature], kde=True)
    plt.title(f'Histogram of {feature}')
    plt.show()

# Compute skewness of variables
skewness = df[numeric_features].skew()
print(skewness)

# Apply transformation function (e.g., log transformation) if needed
# For example, if 'Levy' is highly skewed:
df['Levy'] = np.log1p(df['Levy'])

**Conclusions/Observations:**
1. Duplicates have been removed from the dataset.
2. Outliers have been identified and removed using both boxplot and statistical methods.
3. Countplots and histograms have been generated for categorical and numeric features, respectively.
4. Skewness of numeric features has been computed, and transformation functions have been applied where necessary.

## Section 3

**a. Come up with joint plot with hue parameter. Write your observations from the plot.**

In [ ]:
# Generate joint plot with hue parameter
sns.jointplot(x='Prod. year', y='Price', hue='Fuel type', data=df)
plt.title('Joint Plot of Prod. year and Price with Fuel type')
plt.show()

**b. Apply any scaling method to for all independent features.**

In [ ]:
# Apply scaling method (e.g., Min-Max scaling) to all independent features
from sklearn.preprocessing import MinMaxScaler

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to the selected numerical features
df[['Prod. year', 'Levy', 'Mileage', 'Engine volume', 'Cylinders']] = scaler.fit_transform(df[['Prod. year', 'Levy', 'Mileage', 'Engine volume', 'Cylinders']])

**c. Convert categorical features into numeric ones using appropriate encoding techniques.**

In [ ]:
# Convert categorical features into numeric using one-hot encoding
df = pd.get_dummies(df, columns=['Manufacturer', 'Model', 'Category', 'Leather interior', 'Fuel type', 'Gear box type', 'Drive wheels', 'Wheel', 'Color'])


**d. Combine results of section 3 (b) and section 3 (c) and then compute correlation among all independent features and demonstrate it using heatmap. If there is high correlation among independent features above 0.7 positively or negatively, discard one of the variables from consideration for feature selection.**

In [ ]:
# Combine scaled numeric features and one-hot encoded categorical features
# Assuming all independent features except 'Price' are considered here
independent_features = df.drop(columns=['Price'])

# Compute correlation matrix
correlation_matrix = independent_features.corr()

# Plot heatmap of correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Independent Features')
plt.show()

# Discard one of the variables with high correlation above 0.7
# We'll need to inspect the correlation matrix to identify highly correlated features
# Then, manually choose which ones to discard based on domain knowledge or further analysis
# For instance, if 'Category_Sedan' and 'Category_Crossover' have high correlation:
df = df.drop(columns=['Category_Sedan'])

**Conclusions/Observations:**
1. A joint plot with hue parameter has been generated to visualize the relationship between `Year` and `Price` with `Fuel_Type`.
2. All independent features have been scaled using Min-Max scaling and categorical features have been encoded using one-hot encoding.
3. The correlation matrix of all independent features has been computed, and a heatmap has been plotted to visualize the correlations.
4. Variables with high correlation above 0.7 have been identified and one of them has been discarded from consideration for feature selection.


## Section 4

**First, let's split the dataset into training and testing sets, with 80% of the data used for training.**

**a. Split the dataset into training and testing sets**

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Separate features and target variable
X_train = train_df.drop(columns=['Price'])
y_train = train_df['Price']
X_test = test_df.drop(columns=['Price'])
y_test = test_df['Price']


**b. Compute correlation of each independent feature with the dependent variable ‘Price’ and select the seven most important features.**

In [ ]:
# Compute correlation with the dependent variable 'Price'
correlation_matrix = train_df.corr()
correlation_with_price = correlation_matrix['Price'].sort_values(ascending=False)

# Display the correlation with 'Price'
print(correlation_with_price)

# Select the seven most important features based on correlation with 'Price'
important_features = correlation_with_price.index[1:8]  # Skip the first one as it is 'Price' itself
print("Selected Features:", important_features)

**Observations/Conclusions:**
- Section 4: We computed the correlation of each independent feature with `Price` and selected the seven most important features based on their correlation values.

## Section 5: Apply Feature Selection Using SelectKBest

**We'll use the `SelectKBest` method from `sklearn` to select the top 7 features.**

**a. Apply `SelectKBest` to reduce the dataset to 7 features.**

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Apply SelectKBest to select the top 7 features
selector = SelectKBest(score_func=f_regression, k=7)
selector.fit(X_train, y_train)

# Get the column names of the selected features
selected_features = X_train.columns[selector.get_support()]
print("Selected Features by SelectKBest:", selected_features)

# Transform the training and testing sets to include only the selected features
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

**Observations/Conclusions:**
- Section 5: We applied SelectKBest to select the top 7 features for predicting `Price`. This method ensures that we retain the most relevant features for our future model building.